In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Dense
from tensorflow.keras import Input, Model

# Load CIFAR-10 data
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Preprocess data
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

def resnet_block(X, filters, stride=1):
    F1, F2 = filters

    X_shortcut = X

    # First component of main path
    X = Conv2D(F1, kernel_size=(3, 3), strides=(stride, stride), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(F2, kernel_size=(3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)

    # Shortcut path
    if stride != 1 or X.shape[-1] != F2:
        X_shortcut = Conv2D(F2, kernel_size=(1, 1), strides=(stride, stride), padding='valid')(X_shortcut)
        X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # Add shortcut value to main path
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

# ResNet model
input_shape = (32, 32, 3)
input_X = Input(shape=input_shape)

X = Conv2D(64, (7, 7), strides=(2, 2), padding='same')(input_X)
X = BatchNormalization(axis=3)(X)
X = Activation('relu')(X)

X = resnet_block(X, filters=[64, 64])
X = resnet_block(X, filters=[64, 64])
X = resnet_block(X, filters=[128, 128], stride=2)
X = resnet_block(X, filters=[128, 128])
X = resnet_block(X, filters=[256, 256], stride=2)
X = resnet_block(X, filters=[256, 256])
X = GlobalAveragePooling2D()(X)
X = Dense(10, activation='softmax')(X)

model = Model(inputs=input_X, outputs=X)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=2, batch_size=64, validation_data=(test_images, test_labels))

# Evaluate the model on test data
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {accuracy * 100:.2f}%")

170498071/170498071 [==============================] - 3s 0us/step
Epoch 1/2
316/782 [===========>..................] - ETA: 8:52 - loss: 1.5556 - accuracy: 0.4335